# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rongelap,11.1635,166.8941,29.02,78,8,7.39,MH,1727851331
1,1,bandarbeyla,9.4942,50.8122,29.19,62,6,7.09,SO,1727851332
2,2,waitangi,-43.9535,-176.5597,12.12,86,100,2.68,NZ,1727851333
3,3,grytviken,-54.2811,-36.5092,-6.57,79,21,4.90,GS,1727851335
4,4,la foa,-21.7108,165.8276,22.02,69,5,4.51,NC,1727851336


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    legend='right',
    c='City', 
    cmap='Category10'
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temperature_range = [21.0, 27.0]
ideal_wind_max = 10
cloudness_max = 20.0
city_data_df_ideal = city_data_df[(city_data_df['Max Temp'] >= ideal_temperature_range[0]) & (city_data_df['Max Temp'] <= ideal_temperature_range[1])]
city_data_df_ideal = city_data_df_ideal[city_data_df_ideal['Wind Speed'] <= ideal_wind_max]
city_data_df_ideal = city_data_df_ideal[city_data_df_ideal['Cloudiness'] <= cloudness_max]

# Drop any rows with null values
city_data_df_cleaned = city_data_df_ideal.dropna()

# Display sample data
city_data_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,la foa,-21.7108,165.8276,22.02,69,5,4.51,NC,1727851336
21,21,port mathurin,-19.6833,63.4167,22.62,83,0,8.54,MU,1727851360
23,23,ciudad insurgentes,25.2625,-111.7753,23.25,74,0,2.67,MX,1727851363
54,54,tionk essil,12.7856,-16.5217,24.98,93,13,2.49,SN,1727851402
58,58,metlili chaamba,32.2667,3.6333,25.52,44,0,2.06,DZ,1727851407


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_cleaned[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = hotel_df.apply(lambda _: ' ', axis=1)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,la foa,NC,-21.7108,165.8276,69,
21,port mathurin,MU,-19.6833,63.4167,83,
23,ciudad insurgentes,MX,25.2625,-111.7753,74,
54,tionk essil,SN,12.7856,-16.5217,93,
58,metlili chaamba,DZ,32.2667,3.6333,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    print(latitude)

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
-21.7108
la foa - nearest hotel: Oasis de Tendea
-19.6833
port mathurin - nearest hotel: Escale Vacances
25.2625
ciudad insurgentes - nearest hotel: Hotel Restaurant Californias
12.7856
tionk essil - nearest hotel: Campement Edonia
32.2667
metlili chaamba - nearest hotel: No hotel found
31.9115
canutillo - nearest hotel: Hampton Inn & Suites
-9.1667
aripuana - nearest hotel: No hotel found
37.4423
ermoupolis - nearest hotel: Παλλάδιον
5.6592
mana - nearest hotel: Le Samana
-28.2667
kingscliff - nearest hotel: Kingscliff Beach Hotel
-3.7227
fortaleza - nearest hotel: Hotel Catedral
-20.0167
bowen - nearest hotel: Whitsunday Sands Resort
43.7683
muynoq - nearest hotel: Lala
24.0934
aswan - nearest hotel: Oscar Hotel
-21.2078
avarua - nearest hotel: Paradise Inn
36.9172
dellys - nearest hotel: No hotel found
29.1081
awjilah - nearest hotel: No hotel found
28.629
tazacorte - nearest hotel: App Leyma
29.0744
bani suwayf - nearest hotel: قاعة المؤتمرات
22.9036
bahia hon

,City,Country,Lat,Lng,Humidity,Hotel Name
4,la foa,NC,-21.7108,165.8276,69,Oasis de Tendea
21,port mathurin,MU,-19.6833,63.4167,83,Escale Vacances
23,ciudad insurgentes,MX,25.2625,-111.7753,74,Hotel Restaurant Californias
54,tionk essil,SN,12.7856,-16.5217,93,Campement Edonia
58,metlili chaamba,DZ,32.2667,3.6333,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    legend='right',
    c='City', 
    cmap='Category10',
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)